# COVID -19 MASS GOV

In [1]:
#start session
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2/lib/spark2')
#from pyspark import SparkContext
#from pyspark.sql import SparkSession, SQLContext
from pyspark.sql import SparkSession
from timeit import default_timer as timer
#from modules import claims_sql
from pyspark.sql.functions import col, round

import sys
import os
import pprint
pp = pprint.PrettyPrinter(indent=4)
#DW Netezza Login Information
import getpass
import json
from datetime import date, datetime
import pandas as pd
pd.set_option('display.max_columns', None,'display.max_columns', None)

from IPython.core.interactiveshell import InteractiveShell #display results setting
InteractiveShell.ast_node_interactivity = "all"

# uid = input("Enter LAN ID: ")
# pwd = getpass.getpass("Enter LAN pwd: ")

In [2]:
spark = SparkSession \
        .builder \
        .appName("COVID-19_MassGov") \
        .config("spark.executor.instances", "5") \
        .config("spark.executor.memory", "48G") \
        .config("spark.executor.cores", "5") \
        .getOrCreate()

## COVID-19 MA cases everyday

In [62]:
#download All data from mass.gov
#https://stackoverflow.com/questions/48800385/how-to-download-ms-word-docx-file-in-python-with-raw-data-from-http-url/48811118
import requests

dates = ['3-20','3-21']
dates = ['3-21']
months = ['january','february','march','april','may','june',\
          'july','august','september','october','november','december']

def download_covid_daily_data(dt):
    mth = months[int(dt.split('-')[0])-1]
    dy = dt.split('-')[1]
    f_url = 'https://www.mass.gov/doc/covid-19-cases-in-massachusetts-as-of-'+mth+'-'+str(dy)+'-2020-accessible/download/'
    f_name = 'covid-19-case-report-'+str(dt)+'-2020-acc.docx'
    print(f_url,f_name)
    
    def save_link(book_link, book_name):
        the_book = requests.get(book_link, stream=True)
        with open(book_name, 'wb') as f:
            for chunk in the_book.iter_content(1024 * 1024 * 2):  # 2 MB chunks
                f.write(chunk)

    save_link(f_url,f_name)
    
for day in dates:
    download_covid_daily_data(day)

https://www.mass.gov/doc/covid-19-cases-in-massachusetts-as-of-march-21-2020-accessible/download/ covid-19-case-report-3-21-2020-acc.docx


In [45]:
#https://stackoverflow.com/questions/10366596/how-to-read-contents-of-an-table-in-ms-word-file-using-python

In [4]:
uid = 'abaner02'
sql = 'select * from analytics.w5_covid_19_MASSGOV_'+uid
df = spark.sql(sql).toPandas()
df

,Barnstable,Berkshire,Bristol,Dukes,Essex,Franklin,Hampden,Hampshire,Middlesex,Norfolk,Plymouth,Suffolk,Worcester,Unknown,Total,deaths,gender_Female,gender_Male,hosp_Patientwashospitalized,hosp_Patientwasnothospitalized,hosp_UnderInvestigation,exp_Biogenemployeescontacts,exp_Travelrelated,exp_BerkshireMedicalCentercontacts,exp_Underinvestigation,exp_Biogencontacts,exp_WesternMAcluster,exp_Biogenandhouseholdcontacts,exp_Localtransmission,exp_Biogenconferenceattendeesandhouseholdcontacts,exp_Biogen_Total,test_MAStateLab_positive,test_MAStateLab_total,test_LabCorp_positive,test_LabCorp_total,test_QuestDiagnostics_positive,test_QuestDiagnostics_total,test_Otherlaboratories_positive,test_Otherlaboratories_total,age_lessthan19,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,age_gtthan70,day
0,24,23,24,1,60,2,12,4,199,75,25,126,37,34,646,5,308,338,71,263,312,0,68,0,396,0,0,0,83,99,99,362,3403,103,1651,80,950,101,Notyetavailable,18,93,112,134,119,87,83,3-22
1,11,21,14,1,41,2,9,2,177,69,20,108,24,26,525,1,249,275,61,215,249,0,53,0,306,0,0,0,69,97,97,328,3031,72,1433,55,743,70,Notavailable,0,0,0,0,0,0,0,3-21
2,5,18,6,0,19,1,3,1,119,52,5,72,14,13,328,0,159,169,43,160,125,0,34,0,151,0,0,0,46,97,97,261,2208,29,699,12,225,26,Notavailable,0,0,0,0,0,0,0,3-19
3,9,20,6,0,29,1,3,2,144,64,11,86,19,19,413,1,195,218,58,199,156,0,49,0,204,0,0,0,63,97,97,296,2666,44,940,32,485,41,Notavailable,0,0,0,0,0,0,0,3-20
4,2,17,5,0,14,1,2,0,100,45,5,51,10,4,256,0,119,137,27,151,78,0,26,0,95,0,0,0,38,97,97,224,1743,11,306,12,222,9,Notavailable,0,0,0,0,0,0,0,3-18
5,2,14,5,0,8,0,1,0,89,43,5,42,8,1,218,0,102,116,21,145,52,0,24,0,59,0,0,102,33,0,102,197,1367,1,164,12,220,0,0,0,0,0,0,0,0,0,3-17
6,1,11,2,0,8,0,1,0,83,36,3,36,6,10,197,0,93,104,14,123,60,0,18,0,51,0,0,100,28,0,100,181,1092,1,48,7,156,0,0,0,0,0,0,0,0,0,3-16
7,0,9,0,0,2,0,0,0,60,24,0,26,2,0,123,0,54,69,10,101,12,94,5,8,16,0,0,0,0,0,94,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3-13
8,1,9,1,0,5,0,0,0,65,28,0,27,2,0,138,0,64,74,11,105,22,0,5,0,21,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3-14
9,1,9,1,0,6,0,1,0,75,31,1,31,6,2,164,0,74,90,13,115,36,0,13,0,35,108,8,0,0,0,108,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3-15


In [5]:
import zipfile
import xml.etree.ElementTree

WORD_NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
PARA = WORD_NAMESPACE + 'p'
TEXT = WORD_NAMESPACE + 't'
TABLE = WORD_NAMESPACE + 'tbl'
ROW = WORD_NAMESPACE + 'tr'
CELL = WORD_NAMESPACE + 'tc'

def read_daily_covid_data(dt):
    data_str = ''
    file_x = 'covid-19-case-report-'+dt+'-2020.docx'
    print(file_x)
    with zipfile.ZipFile(file_x) as docx:
        tree = xml.etree.ElementTree.XML(docx.read('word/document.xml'))
    
      
    for table in tree.iter(TABLE):
        for row in table.iter(ROW):
            for cell in row.iter(CELL):
                data_str = data_str + ' '.join(node.text for node in cell.iter(TEXT)) +'\n'
                #print(data_str)
                #print(type(data_str))
    return data_str
                
data = []
#dates = ['3-13','3-14','3-15','3-16','3-17','3-18','3-19','3-20','3-21','3-22','3-23']
dates = ['3-23','3-24']
data_dict_dt = {}
for day in dates:
    data_dict_dt[day] = {}
    curr = read_daily_covid_data(day)
    #print('County --->' , curr.split('County')[1].split('Sex')[0])
    p1 = curr.split('County')
   
    county_raw = [ m.replace(' ','') for m in p1[1].split('Sex')[0].split('\n') if m != '']
    counties = [i[1] for i in enumerate(county_raw) if i[0]%2 == 0]
    counties_data = [i[1] for i in enumerate(county_raw) if i[0]%2 != 0]
    county = dict(list(zip(counties,counties_data)))
    
    gender_raw = p1[1].split('Sex')[1].split('Exposure')[0]
    if gender_raw.find('Age Group') == -1 :
        gender_i = [ m.replace(' ','') for m in gender_raw.split('\n') if m != '']
        age = {}
    else:
        gender_i = [ m.replace(' ','') for m in gender_raw.split('Age Group')[0].split('\n') if m != '']
        # Add Agegroup
        ageraw =gender_raw.split('Age Group')[1]
        ageraw = [ m.replace(' ','') for m in ageraw.split('\n') if m != '']
        age_i = [i[1] for i in enumerate(ageraw) if i[0]%2 == 0]
        age_data = [i[1] for i in enumerate(ageraw) if i[0]%2 != 0]
        age = dict(list(zip(age_i,age_data)))
        
    
    g_i = [i[1] for i in enumerate(gender_i) if i[0]%2 == 0]
    g_data = [i[1] for i in enumerate(gender_i) if i[0]%2 != 0]
    gender = dict(list(zip(g_i,g_data)))
        
     
    p2 = p1[1].split('Sex')[1].split('Exposure')[1]
    
    exposure_raw = p2.split('Hospitalization')[0]
    #print(exposure_raw.find('Deaths'),exposure_raw)
    if exposure_raw.find('Deaths') == -1 :
        x_raw = [ m.replace(' ','') for m in exposure_raw.split('\n') if m != '']
        deaths =0
    else:
        x_raw  = [ m.replace(' ','') for m in exposure_raw.split('Deaths')[0].split('\n') if m != '']
        deaths = int([ m.replace(' ','') for m in exposure_raw.split('Deaths')[1].split('\n') if m != ''][1])
        
    x_i = [i[1] for i in enumerate(x_raw) if i[0]%2 == 0]
    x_data = [i[1] for i in enumerate(x_raw) if i[0]%2 != 0]
    exposure = list(zip(x_i,x_data))
    
    hospitalizations_raw = p2.split('Hospitalization')[1].replace(' ','')
    hospitalizations_raw2 = hospitalizations_raw.split('MAStatePublicHealthLaboratory')[0]
    #print(hospitalizations_raw2)
    h_raw = [ m for m in hospitalizations_raw2.split('\n') if m != '']
    h_i = [i[1] for i in enumerate(h_raw) if i[0]%2 == 0]
    h_data = [i[1] for i in enumerate(h_raw) if i[0]%2 != 0]
    hospitalizations = dict(list(zip(h_i,h_data)))
    
    try:
        p3 = hospitalizations_raw.split('MAStatePublicHealthLaboratory')[1]
        #print('9'* 9)
        ma_lab = p3.split('Laboratory')[0].replace('\n','|')
        ma_lab = ma_lab.split('Patients')[1].split('|')
        ma_lab = [m.replace(' ','') for m in ma_lab if m != '']
        
        lab = p3.split('Laboratory')[1].replace('\n','|')
        lab = lab.split('TotalTested')[1].replace('*','').split('|')
        lab =[m.replace(' ','') for m in lab if m != '']
        #labs = [i[1] for i in enumerate(lab) if i[0]%3 == 0]
        lab = [(i[1],lab[i[0]+1].replace(' ',''),lab[i[0]+2].replace(' ','')) for i in enumerate(lab) if i[0]%3 == 0]
        lab = [('MAStateLab',ma_lab[0],ma_lab[1])] +lab
        lab_dict = {}
        for m in lab:
            lab_dict[m[0]] = {}
            lab_dict[m[0]]['tested_positive'] = m[1]
            lab_dict[m[0]]['tested_total'] = m[2]
        
        lab = lab_dict
    except:
        #print('6'* 9)
        #ma_lab = []
        lab = []
    
    data.append([day,county,gender,exposure,hospitalizations,ma_lab,lab,deaths])
    data_dict_dt[day]['county'] = county
    data_dict_dt[day]['gender'] = gender
    data_dict_dt[day]['exposure'] = exposure
    data_dict_dt[day]['hospitalizations'] = hospitalizations
    data_dict_dt[day]['age'] = age
    data_dict_dt[day]['lab'] = lab
    data_dict_dt[day]['deaths'] = deaths
    
#County data
county_dict = {}
counties = list(data[len(data)-1][1].keys())
for m in counties:
    if m.find('Dukes') != -1:
        m1 = 'Dukes'
    else:
        m1=m
    county_dict[m1] = {}   
    for day,data_y in data_dict_dt.items():
        try:
            county_dict[m1][day] = int(data_y['county'][m])
        except:
            county_dict[m1][day] = 0
time_data = pd.DataFrame(county_dict)
#time_data['Total'] = time_data.sum()
time_data.loc[:,'Total'] = time_data.sum(numeric_only=True, axis=1)

# Deaths
death_d = {}
death_d['deaths'] = {} 
for day,data_x in data_dict_dt.items():
    #print(day,data_x)
    death_d['deaths'][day] = data_x['deaths']
df_death = pd.DataFrame(death_d)

#gender
last_i = list(data_dict_dt.keys())[-1]
gen_list = list(data_dict_dt[last_i]['gender'].keys())
print(gen_list)
g_dict = {}
for m in gen_list:
    g_dict['gender_'+m] = {}
    for day,data_y in data_dict_dt.items():
        try:
            g_dict['gender_'+m][day] = int(data_y['gender'][m])
        except:
            g_dict['gender_'+m][day] = 0
            
g_data = pd.DataFrame(g_dict)

#age
age_list = list(data_dict_dt[last_i]['age'].keys())
print(age_list)
age_dict = {}
for m in age_list:
    m1 = m.replace('yearsofage','').replace('≤','lessthan').replace('≥','gtthan')
    age_dict['age_'+m1] = {}
    for day,data_y in data_dict_dt.items():
        try:
            age_dict['age_'+m1][day] = int(data_y['age'][m])
        except:
            age_dict['age_'+m1][day] = 0
            
age_data = pd.DataFrame(age_dict)

#hospitalizations
hosp_list = list(data_dict_dt[last_i]['hospitalizations'].keys())
print(gen_list)
h_dict = {}
for m in hosp_list:
    h_dict['hosp_'+m] = {}
    for day,data_y in data_dict_dt.items():
        try:
            h_dict['hosp_'+m][day] = int(data_y['hospitalizations'][m])
        except:
            h_dict['hosp_'+m][day] = 0
            
h_data = pd.DataFrame(h_dict)

#lab testin
print(data_dict_dt[last_i]['lab'])
lab_list = list(data_dict_dt[last_i]['lab'].keys())
print(lab_list)
l_dict = {}
for m in lab_list:
    l_dict['test_'+m+'_positive'] = {}
    l_dict['test_'+m+'_total'] = {}
    for day,data_y in data_dict_dt.items():
        try:
            l_dict['test_'+m+'_positive'][day] = data_y['lab'][m]['tested_positive']
            l_dict['test_'+m+'_total'][day] = data_y['lab'][m]['tested_total']
        except:
            l_dict['test_'+m+'_positive'][day] = 0
            l_dict['test_'+m+'_total'][day] = 0
            
l_data = pd.DataFrame(l_dict)

#exposure
exp_list = []
x_dict = {}
for day,data_y in data_dict_dt.items():
    #print(day,data_y['exposure'])
    for m in data_y['exposure']:
        if m[0] not in exp_list:
            exp_list.append(m[0])
#print(exp_list)
for m in exp_list:
    x_dict['exp_'+m.replace('/','').replace('*','')] = {}
    for day,data_y in data_dict_dt.items():
        #print(data_y['exposure'])
        x_dict['exp_'+m.replace('/','').replace('*','')][day] = 0
        for exp in data_y['exposure']:
            #print(m, exp[0],m == exp[0])
            if m == exp[0]:
                #print('HERE',exp[1])
                x_dict['exp_'+m.replace('/','').replace('*','')][day] = int(exp[1])
#print(x_dict)
x_data = pd.DataFrame(x_dict)
biogen_cols =[ m for m in x_data if m.find('Biogen') > -1]
biogen_df = x_data[biogen_cols]
biogen_df['exp_Biogen_Total'] = biogen_df.sum(numeric_only=True, axis=1)
#print(biogen_cols)
    
time_data = time_data.join(df_death).join(g_data)\
        .join(h_data).join(x_data).join(biogen_df.loc[:,'exp_Biogen_Total']).join(l_data).join(age_data)
#time_data
print(time_data)
#time_data.to_csv('covid_19_mass_gov_time_series.csv')
data_dict_dt['3-19']['county']

covid-19-case-report-3-23-2020.docx
covid-19-case-report-3-24-2020.docx
['Female', 'Male', 'Unknown']
['≤19yearsofage', '20-29yearsofage', '30-39yearsofage', '40-49yearsofage', '50-59yearsofage', '60-69yearsofage', '≥70yearsofage', 'Unknown']
['Female', 'Male', 'Unknown']
[]


AttributeError: 'list' object has no attribute 'keys'

In [104]:
#time_data.loc[:,[counties]]
counties_dukes = [m if m.find('Dukes') == -1 else 'Dukes' for m in counties ]
counties_dukes_Transposed = time_data.loc[:,counties_dukes].T
counties_dukes_Transposed.to_csv('county_lvl_covid_19_mass_gov_time_series.csv')
counties_dukes

['Barnstable',
 'Berkshire',
 'Bristol',
 'Dukes',
 'Essex',
 'Franklin',
 'Hampden',
 'Hampshire',
 'Middlesex',
 'Norfolk',
 'Plymouth',
 'Suffolk',
 'Worcester',
 'Unknown']

In [96]:
time_data['test_MAStateLab_positive'] = time_data['test_MAStateLab_positive'].astype('str')
time_data['test_MAStateLab_total'] = time_data['test_MAStateLab_total'].astype('str')
time_data['test_LabCorp_positive'] = time_data['test_LabCorp_positive'].astype('str')
time_data['test_LabCorp_total'] = time_data['test_LabCorp_total'].astype('str')
time_data['test_QuestDiagnostics_positive'] = time_data['test_QuestDiagnostics_positive'].astype('str')
time_data['test_QuestDiagnostics_total'] = time_data['test_QuestDiagnostics_total'].astype('str')
time_data['test_Otherlaboratories_positive'] = time_data['test_Otherlaboratories_positive'].astype('str')
time_data['test_Otherlaboratories_total'] = time_data['test_Otherlaboratories_total'].astype('str')
time_data = time_data.reset_index()
time_data['day'] = time_data['index']
del time_data['index']
print(time_data)
df_mass = spark.createDataFrame(time_data.fillna('0'))

#df_mass.write.mode("overwrite").format('parquet').saveAsTable('analytics.w5_covid_19_MASSGOV_'+uid)

   Barnstable  Berkshire  Bristol  Dukes  Essex  Franklin  Hampden  Hampshire  \
0           0          9        0      0      2         0        0          0   
1           1          9        1      0      5         0        0          0   
2           1          9        1      0      6         0        1          0   
3           1         11        2      0      8         0        1          0   
4           2         14        5      0      8         0        1          0   
5           2         17        5      0     14         1        2          0   
6           5         18        6      0     19         1        3          1   
7           9         20        6      0     29         1        3          2   
8          11         21       14      1     41         2        9          2   
9          24         23       24      1     60         2       12          4   

   Middlesex  Norfolk  Plymouth  Suffolk  Worcester  Unknown  Total  deaths  \
0         60       24        

## COVID-19_MASS GOV Scraping

In [7]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [8]:
url = 'https://www.mass.gov/info-details/covid-19-cases-quarantine-and-monitoring'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [9]:
soup.findAll('a')

[<a class="ma__header__skip-nav visually-hidden focusable" href="#main-content">
     Skip to main content
   </a>, <a href="/" title="Mass.gov home page">
 <img alt="" height="45" src="https://www.mass.gov/libraries/mayflower-artifacts/assets/images/stateseal.png"/>
 <span>Mass.gov</span>
 </a>, <a class="ma__utility-nav__link direct-link" href="https://www.mass.gov/state-a-to-z">
 <svg aria-hidden="true" focusable="false"><use xlink:href="#8cc18f06db89f584a9a8522f018ff661"></use></svg>
 <span>State Organizations</span>
 </a>, <a class="js-clickable-link" href="https://sso.hhs.state.ma.us/">Virtual Gateway <svg aria-hidden="true" focusable="false"><use xlink:href="#8991ca7e135f70607ed0ec9444b44827"></use></svg></a>, <a class="js-clickable-link" href="https://uionline.detma.org/Claimant/Core/Login.ASPX">Unemployment Online <svg aria-hidden="true" focusable="false"><use xlink:href="#8991ca7e135f70607ed0ec9444b44827"></use></svg></a>, <a class="js-clickable-link" href="https://ecse.cse.s

In [10]:
soup.findAll('div')

[<div class="dialog-off-canvas-main-canvas" data-off-canvas-main-canvas="">
 <div class="ma__ajax-pattern js-ajax-site-alerts-jsonapi" data-ma-ajax-endpoint="/jsonapi/node/alert?page[limit]=250&amp;sort=-changed&amp;include=field_target_pages_para_ref,field_alert&amp;filter[status][value]=1&amp;fields[node--alert]=title,changed,entity_url,field_alert_severity,field_alert,field_target_pages_para_ref,field_alert_display&amp;fields[paragraph--emergency_alert]=drupal_internal__id,changed,field_emergency_alert_timestamp,field_emergency_alert_message,field_emergency_alert_link,field_emergency_alert_content&amp;fields[paragraph--target_pages]=field_target_content_ref" data-ma-ajax-render-pattern="@organisms/by-template/emergency-alerts.twig">
 </div>
 <header class="ma__header" id="header" role="banner">
 <a class="ma__header__skip-nav visually-hidden focusable" href="#main-content">
     Skip to main content
   </a>
 <div class="ma__header__container">
 <div class="ma__header__logo">
 <div c

In [30]:
data = soup.findAll(class_='ma__table--responsive__wrapper')[1].find_all('tr')
total = int(str(data[1].find_all('td')[0]).replace('<td>','').replace('</td>',''))
completed = int(str(data[2].find_all('td')[0]).replace('<td>','').replace('</td>',''))
current = int(str(data[3].find_all('td')[0]).replace('<td>','').replace('</td>',''))
data,total,completed,current

([<tr><th>Massachusetts residents subject to COVID-19 quarantine by current status as of March 24, 2020*</th>
  <th> </th>
  </tr>, <tr><th>Total of individuals subject to quarantine</th>
  <td>3802</td>
  </tr>, <tr><th>Total of individuals who have completed monitoring (no longer in quarantine)</th>
  <td>1655</td>
  </tr>, <tr><th>Total of individuals currently undergoing monitoring/under quarantine</th>
  <td>2147</td>
  </tr>],
 3802,
 1655,
 2147)

In [50]:
months = ['january','february','march','april','may','june',\
          'july','august','september','october','november','december']
getcurrdate = str(soup.findAll(class_='ma__table--responsive__wrapper')[0].find('tr'))\
                .replace('<th>COVID-19 cases in Massachusetts','')\
                .replace('<tr>\xa0as of ','').replace('</th>\n<th>\xa0</th>\n</tr>','')
get_dt_str = getcurrdate.replace(',','').split(' ')
mth = get_dt_str[0].lower().strip(' ')
curr_date = str(months.index(mth)+1)+'-'+get_dt_str[1]

getcurrdate,get_dt_str,mth,curr_date

('March 26, 2020', ['March', '26', '2020'], 'march', '3-26')

In [63]:
url = ''
data_d = soup.findAll('a')
for row in data_d:
    if row.text =='Doc' and row['href'].find('/doc/covid-19-cases') != -1:
        url = 'https://www.mass.gov/'+row['href']
        print(url)
        #urllib.request.urlretrieve(url,'/')

https://www.mass.gov//doc/covid-19-cases-in-massachusetts-as-of-march-26-2020-accessible/download


In [65]:
opener = urllib.request.URLopener()
opener.addheader('User-Agent', 'whatever')
filename, headers = opener.retrieve(url, curr_date+'.docx')

In [66]:
# Usafacts website
url = 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
soup.findAll('a')

[<a aria-label="USAFacts back to home" class="h-full flex items-center text-usafacts-blue" href="/">
 <svg class="header__brand fill-current"><use xlink:href="#brand"></use></svg>
 <span class="hidden font-bold text-3xl tracking-tight">USAFacts</span>
 </a>,
 <a class="border-gray-900 border-t-4 text-gray-900 inline-block mt-0 font-base font-bold uppercase tracking-widest text-sm -mt-1 mx-4 2xl:mx-5 first:ml-0 last:mr-0 py-10 has-dropdown" data-content="dropmenu--issues" href="/issues/">Issues</a>,
 <a class="border-gray-900 border-t-4 text-gray-900 inline-block mt-0 font-base font-bold uppercase tracking-widest text-sm -mt-1 mx-4 2xl:mx-5 first:ml-0 last:mr-0 py-10 has-dropdown" data-content="dropmenu--data" href="/data/">Data</a>,
 <a class="border-gray-900 border-t-4 text-gray-900 inline-block mt-0 font-base font-bold uppercase tracking-widest text-sm -mt-1 mx-4 2xl:mx-5 first:ml-0 last:mr-0 py-10 has-dropdown" data-content="dropmenu--reports" href="/reports/">Reports</a>,
 <a class

In [68]:
conf_url = ''
death_url = ''
data_d = soup.findAll('a')
for row in data_d:
    #if row.text =='Doc' and row['href'].find('/doc/covid-19-cases') != -1:
    if row.text =='Confirmed Cases':
        conf_url = row['href']
        print(conf_url)
    if row.text =='Deaths':
        death_url = row['href']
        print(death_url)

https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv
https://static.usafacts.org/public/data/covid-19/covid_deaths_usafacts.csv


In [69]:
s = pd.Series([2, 2, 3, 4, 5])
s.diff()

0    NaN
1    0.0
2    1.0
3    1.0
4    1.0
dtype: float64

In [76]:
p = pd.DataFrame([[2],[2],[3],[4],[5]],columns=['val'])
p['diff'] = p['val'].diff()
p

,val,diff
0,2,NaN
1,2,0.0
2,3,1.0
3,4,1.0
4,5,1.0


In [80]:
dictx = {   '3-09': {   'county': {   'Berkshire': '5',
                              'Middlesex': '15',
                              'Norfolk': '10',
                              'Suffolk': '10',
                              'Worcester': '1'}},
    '3-10': {   'county': {   'Berkshire': '7',
                              'Essex': '1',
                              'Middlesex': '41',
                              'Norfolk': '22',
                              'Suffolk': '20',
                              'Worcester': '1'}},
    '3-11': {   'county': {   'Berkshire': '7',
                              'Essex': '1',
                              'Middlesex': '44',
                              'Norfolk': '23',
                              'Suffolk': '19',
                              'Worcester': '1'}},
    '3-12': {   'county': {   'Berkshire': '9',
                              'Essex': '2',
                              'Middlesex': '49',
                              'Norfolk': '24',
                              'Suffolk': '22',
                              'Unknown': '1',
                              'Worcester': '1'}},
    '3-13': {   'county': {   'Berkshire': '9',
                              'Essex': '2',
                              'Middlesex': '60',
                              'Norfolk': '24',
                              'Suffolk': '26',
                              'Worcester': '2'}},
    '3-14': {   'county': {   'Barnstable': '1',
                              'Berkshire': '9',
                              'Bristol': '1',
                              'Essex': '5',
                              'Middlesex': '65',
                              'Norfolk': '28',
                              'Suffolk': '27',
                              'Worcester': '2'}},
    '3-15': {   'county': {   'Barnstable': '1',
                              'Berkshire': '9',
                              'Bristol': '1',
                              'Essex': '6',
                              'Hampden': '1',
                              'Middlesex': '75',
                              'Norfolk': '31',
                              'Plymouth': '1',
                              'Suffolk': '31',
                              'Unknown': '2',
                              'Worcester': '6'}},
    '3-16': {   'county': {   'Barnstable': '1',
                              'Berkshire': '11',
                              'Bristol': '2',
                              'Essex': '8',
                              'Hampden': '1',
                              'Middlesex': '83',
                              'Norfolk': '36',
                              'Plymouth': '3',
                              'Suffolk': '36',
                              'Unknown': '10',
                              'Worcester': '6'}},
    '3-17': {   'county': {   'Barnstable': '2',
                              'Berkshire': '14',
                              'Bristol': '5',
                              'Essex': '8',
                              'Hampden': '1',
                              'Middlesex': '89',
                              'Norfolk': '43',
                              'Plymouth': '5',
                              'Suffolk': '42',
                              'Unknown': '1',
                              'Worcester': '8'}},
    '3-18': {   'county': {   'Barnstable': '2',
                              'Berkshire': '17',
                              'Bristol': '5',
                              'Essex': '14',
                              'Franklin': '1',
                              'Hampden': '2',
                              'Middlesex': '100',
                              'Norfolk': '45',
                              'Plymouth': '5',
                              'Suffolk': '51',
                              'Unknown': '4',
                              'Worcester': '10'}},
    '3-19': {   'county': {   'Barnstable': '5',
                              'Berkshire': '18',
                              'Bristol': '6',
                              'Essex': '19',
                              'Franklin': '1',
                              'Hampden': '3',
                              'Hampshire': '1',
                              'Middlesex': '119',
                              'Norfolk': '52',
                              'Plymouth': '5',
                              'Suffolk': '72',
                              'Unknown': '13',
                              'Worcester': '14'}},
    '3-20': {   'county': {   'Barnstable': '9',
                              'Berkshire': '20',
                              'Bristol': '6',
                              'Essex': '29',
                              'Franklin': '1',
                              'Hampden': '3',
                              'Hampshire': '2',
                              'Middlesex': '144',
                              'Norfolk': '64',
                              'Plymouth': '11',
                              'Suffolk': '86',
                              'Unknown': '19',
                              'Worcester': '19'}},
    '3-21': {   'county': {   'Barnstable': '11',
                              'Berkshire': '21',
                              'Bristol': '14',
                              'DukesandNantucket': '1',
                              'Essex': '41',
                              'Franklin': '2',
                              'Hampden': '9',
                              'Hampshire': '2',
                              'Middlesex': '177',
                              'Norfolk': '69',
                              'Plymouth': '20',
                              'Suffolk': '108',
                              'Unknown': '26',
                              'Worcester': '24'}},
    '3-22': {   'county': {   'Barnstable': '24',
                              'Berkshire': '23',
                              'Bristol': '24',
                              'DukesandNantucket': '1',
                              'Essex': '60',
                              'Franklin': '2',
                              'Hampden': '12',
                              'Hampshire': '4',
                              'Middlesex': '199',
                              'Norfolk': '75',
                              'Plymouth': '25',
                              'Suffolk': '126',
                              'Unknown': '34',
                              'Worcester': '37'}},
    '3-23': {   'county': {   'Barnstable': '30',
                              'Berkshire': '26',
                              'Bristol': '25',
                              'DukesandNantucket': '1',
                              'Essex': '73',
                              'Franklin': '2',
                              'Hampden': '15',
                              'Hampshire': '6',
                              'Middlesex': '232',
                              'Norfolk': '82',
                              'Plymouth': '32',
                              'Suffolk': '154',
                              'Unknown': '57',
                              'Worcester': '42'}},
    '3-24': {   'county': {   'Barnstable': '40',
                              'Berkshire': '37',
                              'Bristol': '31',
                              'DukesandNantucket': '2',
                              'Essex': '118',
                              'Franklin': '5',
                              'Hampden': '24',
                              'Hampshire': '8',
                              'Middlesex': '304',
                              'Norfolk': '129',
                              'Plymouth': '64',
                              'Suffolk': '234',
                              'Unknown': '90',
                              'Worcester': '73'}},
    '3-25': {   'county': {   'Barnstable': '51',
                              'Berkshire': '71',
                              'Bristol': '67',
                              'DukesandNantucket': '3',
                              'Essex': '177',
                              'Franklin': '14',
                              'Hampden': '45',
                              'Hampshire': '11',
                              'Middlesex': '446',
                              'Norfolk': '222',
                              'Plymouth': '101',
                              'Suffolk': '342',
                              'Unknown': '159',
                              'Worcester': '129'}},
    '3-26': {   'county': {   'Barnstable': '67',
                              'Berkshire': '73',
                              'Bristol': '90',
                              'DukesandNantucket': '3',
                              'Essex': '247',
                              'Franklin': '16',
                              'Hampden': '55',
                              'Hampshire': '17',
                              'Middlesex': '538',
                              'Norfolk': '292',
                              'Plymouth': '138',
                              'Suffolk': '448',
                              'Unknown': '267',
                              'Worcester': '166'}}}

dictx

{'3-09': {'county': {'Berkshire': '5',
   'Middlesex': '15',
   'Norfolk': '10',
   'Suffolk': '10',
   'Worcester': '1'}},
 '3-10': {'county': {'Berkshire': '7',
   'Essex': '1',
   'Middlesex': '41',
   'Norfolk': '22',
   'Suffolk': '20',
   'Worcester': '1'}},
 '3-11': {'county': {'Berkshire': '7',
   'Essex': '1',
   'Middlesex': '44',
   'Norfolk': '23',
   'Suffolk': '19',
   'Worcester': '1'}},
 '3-12': {'county': {'Berkshire': '9',
   'Essex': '2',
   'Middlesex': '49',
   'Norfolk': '24',
   'Suffolk': '22',
   'Unknown': '1',
   'Worcester': '1'}},
 '3-13': {'county': {'Berkshire': '9',
   'Essex': '2',
   'Middlesex': '60',
   'Norfolk': '24',
   'Suffolk': '26',
   'Worcester': '2'}},
 '3-14': {'county': {'Barnstable': '1',
   'Berkshire': '9',
   'Bristol': '1',
   'Essex': '5',
   'Middlesex': '65',
   'Norfolk': '28',
   'Suffolk': '27',
   'Worcester': '2'}},
 '3-15': {'county': {'Barnstable': '1',
   'Berkshire': '9',
   'Bristol': '1',
   'Essex': '6',
   'Hampden': 

In [81]:
sorted(dictx)

['3-09',
 '3-10',
 '3-11',
 '3-12',
 '3-13',
 '3-14',
 '3-15',
 '3-16',
 '3-17',
 '3-18',
 '3-19',
 '3-20',
 '3-21',
 '3-22',
 '3-23',
 '3-24',
 '3-25',
 '3-26']

In [82]:
dates = list(sorted(dictx.keys()))
dates

['3-09',
 '3-10',
 '3-11',
 '3-12',
 '3-13',
 '3-14',
 '3-15',
 '3-16',
 '3-17',
 '3-18',
 '3-19',
 '3-20',
 '3-21',
 '3-22',
 '3-23',
 '3-24',
 '3-25',
 '3-26']